In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

from bo_functions_generic import gen_theta_set

from CS2_bo_plotters import plot_obj_abs_min
from CS2_bo_plotters import value_plotter
from CS2_bo_plotters import plot_obj
from CS2_bo_plotters import plot_Theta
from CS2_bo_plotters import plot_Theta_min
from CS2_bo_plotters import path_name
from CS2_bo_functions_multi_dim import set_ep
from CS2_bo_plotters import plot_org_train, plot_EI_abs_max

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html

def csv_to_array(csv_path):
    """
    Turn a csv file into a numpy array so that it can be used with bo_plotters
    
    Parameters:
    -----------
        csv_path: str, path of csv you want to turn into an array
    Returns:
    --------
        csv_array: np.ndarray, array on values in the CSV file
    """
    
    csv_pd = pd.read_csv(csv_path, index_col = False)
    csv_pd.drop(columns=csv_pd.columns[0], 
        axis=1, 
        inplace=True)
    csv_array = csv_pd.to_numpy()
    
    return csv_array

In [3]:
# DateTime = "2022/10/21/13-05" #Date and Time - Dispayed in job file
DateTime = "2022/10/21/13-18" #Date and Time - Dispayed in job file
# DateTime = None

emulator = False
obj = "obj"
sparse_grid = False
sep_fact = 0.1

exp_d = 1
n = 5

param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
indecies = [list(param_dict.keys())[list(param_dict.values()).index('\\theta_1')],
            list(param_dict.keys())[list(param_dict.values()).index('\\theta_2')]]

Theta_True = np.array([1,-1])

bounds = np.array([[-2, -2],
                   [ 2,  2]])


# exp_d = 2
# n = 15 #Number of experimental data points to use

# param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
#               4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}
# indecies = [list(param_dict.keys())[list(param_dict.values()).index('a_1')],
#             list(param_dict.keys())[list(param_dict.values()).index('a_2')]]

# Constants = np.array([[-200,-100,-170,15],
#                       [-1,-1,-6.5,0.7],
#                       [0,0,11,0.6],
#                       [-10,-10,-6.5,0.7],
#                       [1,0,-0.5,-1],
#                       [0,0.5,1.5,1]])

# Theta_True = Constants[1:3].flatten()

# bounds = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
#                    [ 2,  2,   0,  2,  2,  2, 15,  2]])

param_names_list = [param_dict[0], param_dict[1]] #Fill in depending on problem
mesh_combo = str(param_names_list[0]) + "-" + str(param_names_list[1])

mesh_combo = None
# run = 1
# Bo_iter = 1

run = 12
Bo_iter = 6

set_lengthscale = None
save_CSV = False

t = 20
if emulator == True:
    t = n*t

#Pull Experimental data from CSV
exp_data_doc = 'Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]

#Define GP Testing space
p=20
d = len(Theta_True)


# Need to redefine to save theta_set_org 
# theta_set_org = gen_theta_set(LHS = True, n_points = p, dimensions = d, bounds = bounds)
# Theta1_lin = np.linspace(np.min(theta_set_org[:,indecies[0]]),np.max(theta_set_org[:,indecies[0]]), p)
# Theta2_lin = np.linspace(np.min(theta_set_org[:,indecies[1]]),np.max(theta_set_org[:,indecies[1]]), p)
# theta_mesh = np.array(np.meshgrid(Theta1_lin, Theta2_lin)) 

theta_mesh = gen_theta_set(LHS = False, n_points = p, dimensions = d, bounds = bounds)
theta_mesh = theta_mesh.T.reshape((2, p, p))
# print(theta_mesh.shape)
    
ep = set_ep(emulator, obj, sparse_grid)
# ep = 1

# tot_iter = bo_iters= 3 #Total Number of iters - Dispayed in job file
# tot_runs = 2 #Total Number of runs - Dispayed in job file

tot_iter = bo_iters= 100 #Total Number of iters - Dispayed in job file
tot_runs = 15 #Total Number of runs - Dispayed in job file

is_figure = False
save_figure = False

run -= 1 # run should be number you want -1
Bo_iter -= 1 #Iter you want -1

In [6]:
#WORKS

csv_end = ".csv" #For obj
fxn = "plot_obj"

median_obj =[]
sep_fact_list = np.linspace(0.1,1,10)
for k in range(len(sep_fact_list)):  
    path = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
          title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                 sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    # print(path)
    obj_array = csv_to_array(path)
    all_obj_list = []
    for i in range(tot_runs): 
        #Plot data
        obj_df_run = pd.DataFrame(data = obj_array[i])
        obj_df_i = obj_df_run.loc[(abs(obj_df_run) > 1e-6).any(axis=1),0]
        obj_no_zero_arr = list(np.array(obj_df_i).flatten())
        all_obj_list += obj_no_zero_arr
    median_obj.append(np.median(all_obj_list))

median_obj = np.array(median_obj)
median_obj

array([-0.06883678,  1.07129992,  2.71002411,  2.38755975,  2.46067209,
        2.47833052,  2.49371024,  2.61335076,  2.49591158,  2.41574054])

In [7]:
#WORKS

sep_fact_list = np.linspace(0.1,1,10)
med_iter_list = []
sse_best_list = []
iter_best_num_list = []
iter_median_num_list = []
run_best_num_list = []
for k in range(len(sep_fact_list)):  
    csv_end = ".csv" #For obj
    fxn = "plot_obj"

    path = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
              title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                     sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    # print(path)
    obj_abs_min = csv_to_array(path)

    iter_max = []
    run_best = []
    iter_best = []
    sse_best = []
    for i in range(tot_runs):
        obj_min_df_run = pd.DataFrame(data = obj_abs_min[i])
        obj_min_df_i = obj_min_df_run.loc[(abs(obj_min_df_run) > 1e-6).any(axis=1),0]
        sse_best.append(np.amin(obj_min_df_i))
        iter_max.append(len(obj_min_df_i))
        min_iter = np.where(np.amin(obj_min_df_i) == obj_min_df_i) 
        iter_best.append(min_iter[0][0])
        run_best.append(i)
#     print(iter_max)    
    iter_median_num_list.append(np.median(iter_max))
    iter_best_act = np.array(iter_best) + 1
    run_best_act = np.array(run_best) + 1 
    sse_best_num = np.amin(sse_best)
    sse_best_list.append(sse_best_num)
    iter_best_ind = sse_best.index(sse_best_num)
    run_best_list = run_best_act[iter_best_ind]
    iter_best_num = iter_best_act[iter_best_ind]
    median_iters = np.median(iter_best_act)
    med_iter_list.append(median_iters)
    iter_best_num_list.append(iter_best_num)
    run_best_num_list.append(run_best_list)
    
print(med_iter_list)
print(sse_best_list)
print(iter_best_num_list)
print(iter_median_num_list)
print(run_best_num_list)
    

[3.0, 2.0, 1.0, 2.0, 3.0, 8.0, 10.0, 100.0, 100.0, 100.0]
[-9.60184037645952, -0.6937634501690625, -3.858615575249479, 1.4599631359803595, 1.5979976268395064, 0.3126564517372351, 1.484999392854518, 2.330422647270385, 2.3476312035673157, 2.3980914142756293]
[45, 8, 28, 100, 100, 100, 1, 100, 100, 100]
[100.0, 4.0, 3.0, 5.0, 6.0, 19.0, 100.0, 100.0, 100.0, 100.0]
[3, 7, 3, 7, 7, 7, 7, 8, 11, 13]


In [8]:
sep_fact_list = np.linspace(0.1,1,10)
theta_Opt = np.zeros((len(sep_fact_list),d))

for k in range(len(sep_fact_list)):
    Theta_array = []
    fxn = "plot_Theta"
    for i in range(len(param_dict.keys())):
        csv_end = "/"+param_dict[i]+ ".csv" #For Thetas #Need to ensure this saves correctly

        path_i = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
                  title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                         sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = csv_end)
        path_i = path_i.replace("Param_Conv/\\theta_"+str(i+1), "Theta_Conv/Theta_Conv_"+str(i+1))
#         print(path_i)
        Theta_array.append( csv_to_array(path_i) )

    Theta_array = np.array(Theta_array)
    Theat_Opt = Theta_array[:, iter_best_num_list[0]-1, run_best_num_list[0]-1]
    theta_Opt[k] = Theat_Opt
print(theta_Opt)

[[ 0.99477801 -1.00431648]
 [ 0.34728607 -0.94279038]
 [ 0.94871834 -0.90441752]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 1.13140187 -1.61735485]
 [ 1.04200026 -1.57767536]]


In [10]:
#Create table arrays using pandas
df = pd.DataFrame(list(zip(theta_Opt[:,0], theta_Opt[:,1], sse_best_list, median_obj,iter_best_num_list,iter_median_num_list)),
               columns =['Theta 1', 'Theta 2', 'Best SSE', "Median SSE", "Best Eval #", "Median Eval #"])

In [11]:
print(df)

    Theta 1   Theta 2  Best SSE  Median SSE  Best Eval #  Median Eval #
0  0.994778 -1.004316 -9.601840   -0.068837           45          100.0
1  0.347286 -0.942790 -0.693763    1.071300            8            4.0
2  0.948718 -0.904418 -3.858616    2.710024           28            3.0
3  0.000000  0.000000  1.459963    2.387560          100            5.0
4  0.000000  0.000000  1.597998    2.460672          100            6.0
5  0.000000  0.000000  0.312656    2.478331          100           19.0
6  0.000000  0.000000  1.484999    2.493710            1          100.0
7  0.000000  0.000000  2.330423    2.613351          100          100.0
8  1.131402 -1.617355  2.347631    2.495912          100          100.0
9  1.042000 -1.577675  2.398091    2.415741          100          100.0
